In [ ]:
import os
import torch
import csv
from tqdm import tqdm
from torch.utils.data import random_split,DataLoader, Subset
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torchvision.models import mobilenet_v3_large
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from collections import Counter

In [ ]:
data_dir = "/kaggle/input/food41/images"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor(),         
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataset

In [ ]:
indices = list(range(len(dataset)))

train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42, stratify=dataset.targets)

In [ ]:
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

In [ ]:
val_ratio = 0.5 
val_size = int(len(test_dataset) * val_ratio)
test_size = len(test_dataset) - val_size
val_dataset, test_dataset = random_split(test_dataset, [val_size, test_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8)

print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")
print(f"Testing images: {len(test_dataset)}")

Training images: 80800
Validation images: 10100
Testing images: 10100


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("/kaggle/working/best_model.pth", map_location=device)
model = checkpoint['model']  
class_names = checkpoint['classes'] 
model = model.to(device)  
model.eval()  

def evaluate_model(data_loader):
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

test_preds, test_labels = evaluate_model(test_loader)

print("Classification Report:")
print(classification_report(test_labels, test_preds, target_names=class_names))

test_accuracy = accuracy_score(test_labels, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")